# A3 - OpenJij core interface 実践

前回ではスピン数$N=5$の小さいシステムでのデモンストレーションを行いましたが、この章ではさらに大規模な計算を通してOpenJij core interfaceの機能、使い方をより詳しく見ていきます。

題材としてランダムイジング模型を取り上げます。この模型は、ハミルトニアンが
\begin{align}
H &= \sum_{i<j}J_{ij}\sigma_i \sigma_j + \sum_{i=1}^{N}h_i \sigma_i \\
\sigma_i &= \pm 1 (i=1 \cdots N)
\end{align}
で与えられて、$J_{ij}, h_{i}$が$(i,j)$に対して様々な値を持つ模型です。
様々な種類の最適化問題がこのような模型の一番低いエネルギーを求める問題となりますが、エネルギーの最も低い状態を実現する$\sigma_i$を計算するには一般に非常に難しいです。

### 問題の定義 - Graph -

問題を定義してみましょう。`graph`の中の**Dense**モジュールが必要となります。
> `graph`中には基本的に**Dense**モジュールと**Sparse**モジュールがあり、今回の例のように全ての$J_{ij}$で値が定義されている場合には**Dense**モジュールが適しています。多くの$J_{ij}$の要素が0の場合には**Sparse**が適しています。使い方はどちらもほぼ変わりません。

In [48]:
import cxxjij.graph as G
#問題サイズを100とします。
N = 100

graph = G.Dense(N)
#sparseの場合
#graph = G.Sparse(N)

$J_{ij}, h_i$を設定します。今回は平均0、標準偏差1のGauss分布から生成される値を設定します。

In [49]:
!pip install numpy #乱数生成にnumpyを使います。

Defaulting to user installation because normal site-packages is not writeable


In [50]:
import numpy as np
mu, sigma = 0, 1

for i in range(N):
    for j in range(N):
        #Jijの値が大きくなりすぎてしまうので、全体の係数を1/Nしています。
        graph[i,j] = 0 if i == j else np.random.normal()/N

for i in range(N):
    graph[i] = np.random.normal()/N

ここで、縦磁場に関しては、`graph[i]`でも、`graph[i,i]`でもどちらでもアクセスできます。

また、イジングモデルの定義上、$J_{ij}$と$J_{ji}$は自動で同じ値となります。

In [54]:
graph[20] = 0.5
print(graph[20,20])
print(graph[20])
graph[12,34] = -0.6
print(graph[12,34])
print(graph[34,12])

0.5
0.5
-0.6
-0.6


**Dense** (および**Sprase**)は以下のメソッドを持っています。

- .gen_spin(seed)
    乱数生成のためのseedを与えると、システムサイズ (ここでは$N=100$)分のスピン列を出力します。
    seedがない場合にはランダムにseedが決定されます。
- .size()
    問題のサイズ($N$)を表示します。
- .adj_nodes(i)
    インデックス$i$に隣接するノード一覧を表示します。
- .calc_energy(spin)
    スピン列spinを与えられた際に、エネルギー
    \begin{align}
    H &= \sum_{i<j}J_{ij}\sigma_i \sigma_j + \sum_{i=1}^{N}h_i \sigma_i
    \end{align}
    を計算します。